# Lab 12: Classification

Please complete this lab by providing answers in cells after the question. Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

In [ ]:
import numpy as np
from datascience import *

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

This lab will cover the main steps for applying machine learning models.
- Create train and test sets
- Fit the models using train set
- Predict using the test set
- Evaluate models using metrics such as precision and recall
- Make your conclusions

## Brazilian Sign Language

Brazilian Sign Language is a visual language used primarily by Brazilians who are deaf.  It is more commonly called Libras.  People who communicate with visual language are called *signers*. 

Programs like Siri or Google Now begin the process of understanding human speech by classifying short clips of raw sound into basic categories called *phones*.  For example, the recorded sound of someone saying the word "robot" might be broken down into several phones: "rrr", "oh", "buh", "aah", and "tuh".  Phones are then grouped together into further categories like words ("robot") and sentences ("I, for one, welcome our new robot overlords") that carry more meaning.

A visual language like Libras has an analogous structure.  Instead of phones, each word is made up of several *hand movements*.  As a first step in interpreting Libras, we can break down a video clip into small segments, each containing a single hand movement.  The task is then to figure out what hand movement each segment represents.

We can do that with classification!

The [data](https://archive.ics.uci.edu/ml/machine-learning-databases/libras/movement_libras.names) in this exercise come from Dias, Peres, and Biscaro, researchers at the University of Sao Paulo in Brazil.  They identified 15 distinct hand movements in Libras (probably an oversimplification, but a useful one) and captured short videos of signers making those hand movements.  (You can read more about their work [here](http://ieeexplore.ieee.org/Xplore/login.jsp?url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F5161636%2F5178557%2F05178917.pdf&authDecision=-203). The paper is gated, so you will need to use the UMD Wi-Fi or VPN to access it.)

For each video, they chose 45 still frames from the video and identified the location (in horizontal and vertical coordinates) of the signer's hand in each frame.  Since there are two coordinates for each frame, this gives us a total of 90 numbers summarizing how a hand moved in each video.  Those 90 numbers will be our *attributes*.

Each video is *labeled* with the kind of hand movement the signer was making in it.  Each label is one of 15 strings like "horizontal swing" or "vertical zigzag".

For simplicity, we're going to focus on distinguishing between just two kinds of movements: "horizontal straight-line" and "vertical straight-line".  We took the Sao Paulo researchers' original dataset, which was quite small, and used some simple techniques to create a much larger synthetic dataset.

These data are in the file `movements.csv`.  Run the next cell to load it.

In [ ]:
movements = Table.read_table("movements.csv")
movements.show(5)

### Step 1: Create train and test sets

First, let's split up the data into train and test sets. For this assignment, we will do a simple holdout set, assigning a random 20% of the data as the test data, and building the model on the remaining 80% of the data. 

<font color = 'red'>**Question 1. Create two Tables, one called `test` and one called `train`. The `test` table should contain a random 20% of the data, while the `train` Table should contain the other 80%.**<\font>

*Hint:* You can shuffle the entire dataset (sample the whole dataset without replacement), then just take the top 80% as your train data.

In [ ]:
# Find the number of rows you want to take by multiplying the number of rows in
# movements by 0.8. Remember, this needs to be an integer!
rows_to_take = int(...)

# Shuffle the movements Table
shuffled_movements = ...

# Use .take and np.arange to split the data into train and test.
# train should be the first rows_to_take rows of shuffled_movements
# test should be the rest
train = ...
test = ...

Make sure you have 768 rows in the train set and 192 rows in the test set.

In [ ]:
train.num_rows

In [ ]:
test.num_rows

### Step 2: Fitting the models

Now, let's use the `train` data to fit a few different K-Nearest Neighbors models. We can fit the a k-Nearest Neighbors model by creating the model object, then fitting it with the appropriate predictors and outcome. An example with k = 3 is provided below.

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)

predictors = train.drop('Movement type').rows
outcome = train.column('Movement type')

knn.fit(X = predictors, y = outcome)

<font color = 'red'>**Question 2. Create three model objects called `knn5`, `knn9`, and `knn15` that represent the K-Nearest Neighbors classifiers with k = 5, 9, and 15, respectively. Using the `train` Table you created above, fit each of the three models. Use all of the Frame x and y variables as your features (predictors) to predict the Movement type.**</font>

If you're not sure about the exact format of the data needed, remember that you need to use `.rows` for the `X` values and `.column` for the `y` values.

In [ ]:
# Create the model objects
knn5 = ...
knn9 = ...
knn15 = ...

predictors = ...
outcome = ...

...

After you fit the models, you can use the `.classes_` instance variable to check the classes that are being predicted. This shows the categories that are being predicted, as well as the order of them when we make predictions. 

In [ ]:
knn.classes_

Note which class is the "0" class (the first category) and which is the "1" class (the second category). We will use this information later on when getting predictions. 

### Step 3: Predict Test Set

We can use the `.predict` method to predict the outcome using our model object. This uses the default of 0.5 as the **threshold** for a prediction. That is, it makes a prediction of "1" if more than 50% of the closest nearest neighbors are a "1". However, there are some cases in which we might want to predict a "1" even if it's a bit less than 50%. For example, if we want to detect low birthweight babies, we might want to lower the threshold a little bit to make sure we catch more low birthweight babies. If 2 out of 5 of the closest neighbors were low birthweight, we might still want to predict that birth to be low birthweight since it seems like it is a relatively higher risk.  

In order to be able to use different thresholds, we can use the `.predict_proba` method with the model objects. The `.predict_proba` method provides scores for each class. For the K-Nearest Neighbors algorithm, it provides a score based on the "votes" of the nearest neighbors as a proportion. 

As part of this, we need to select one of the two classes as the "positive" or "1" outcome. Since there isn't one outcome we're particularly interested in out of the two possibilities, we will just choose "horizontal straight-line" to be positive.

> In other situations, there might be one that you care more about. For example, if you are trying to predict recidivism in order to provide social programs to help released prisoners at higher risk of returning, then your "positive" outcome -- the outcome you are trying to predict -- would be "return", so that is what you would want to set as `True` below. 

We can't simply evaluate using the scores, though. We need to set a threshold so that we predict that an observation is "horizontal straight-line" or not. Here, we will set the threshold to be 0.5, but remember, there is nothing special about this number. We could have easily selected any other number between 0 and 1, and we should, in practice, try a wide range of values and see how our models perform with each of these thresholds.

An example using K-Nearest Neighbors is shown below.

In [ ]:
# Setting a threshold (above means predicted to be horizontal straight-line)
threshold = 0.3

Since `horizontal straight-line` was the "0" case (see the `knn5.classes_` that we ran above), we need to make sure we get only the "0" scores. We do this using the `[:,0]` code below after we use `.predict_proba`. We then compare the scores to the `threshold` value to get our predictions.

In [ ]:
# Make sure you fit the model before running this!
test_features = test.drop('Movement type').rows
knn_predicted = knn.predict_proba(test_features)[:,0] > threshold
knn_predicted

The `True` and `False` values correspond to a prediction of `horizontal straight-line` (for `True`) and `vertical straight-line` (for `False`). This is because we set `horizontal straight-line` to be our positive case by using `[:,0]`. If we had used `[:,1]`, it would have been the other way around, with `True` being a prediction of `vertical straight-line`. 

<font color = 'red'>**Question 3. Create an array called `expected` that contain the expected values based on the test set. The `expected` array should contain `True` if the observation is actually a "horizontal straight-line" and `False` otherwise. Note that the `expected` array is based entirely on the real dataset, and not on any predictions! Create additional arrays that contain the predicted values for each of the models that we've fit (call them `knn5_predicted`, `knn9_predicted`, and `knn15_predicted`).**</font>

*Hint:* The `expected` array should take the `Movement type` column in the `test` Table and compare it to `horizontal straight-line` to get an array of `True` and `False` values. 

In [ ]:
expected = ...

knn5_predicted = ...
knn9_predicted = ...
knn15_predicted = ...


### Step 4: Evaluate

You can get a confusion matrix using the `confusion_matrix` function that we brought in at the beginning. This is part of the `sklearn.metrics` module.

In [ ]:
conf_matrix = confusion_matrix(expected,knn_predicted)

In [ ]:
conf_matrix

The columns represent predictions and the rows represent actual values, so the top left is **true negatives (TN)**, the bottom right is **true positives (TP)**, the top right is **false positives (FP)**, and the bottom left is **false negatives (FN)**.

<img src="confusion_matrix.jpeg"/>

#### Evaluation metrics

We've already gone over using the `accuracy_score` function to find the accuracy of a prediction model.

In [ ]:
accuracy_score(expected,knn_predicted)

Two metrics that are often more relevant than overall accuracy are **precision** and **recall**. 

Precision measures the accuracy of the classifier when it predicts an example to be positive. It is the ratio of correctly predicted positive examples to examples predicted to be positive. 

$$ Precision = \frac{TP}{TP+FP}$$

Recall measures the accuracy of the classifier to find positive examples in the data. 

$$ Recall = \frac{TP}{TP+FN} $$

By selecting different thresholds we can vary and tune the precision and recall of a given classifier. A conservative classifier (threshold 0.99) will classify a case as 1 only when it is *very sure*, leading to high precision. On the other end of the spectrum, a low threshold (e.g. 0.01) will lead to higher recall. 

We can use the `precision_score` and `recall_score` functions to find the value of these measures.

In [ ]:
precision_score(expected,knn_predicted)

In [ ]:
recall_score(expected,knn_predicted)

<font color = 'red'>**Question 4. Find the confusion matrix for one of the other models (k = 5, 9, or 15). Use the `accuracy_score`, `precision_score`, `recall_score` functions to find accuracy, precision, and recall for your models.**</font>

<font color = 'red'>**Question 5. Which model of two we measured the performance of above (k = 3 and whichever you chose in question 4) performed the best according to accuracy? What about precision? Recall?**</font>

### Step 5: Repeating the steps

We've done one iteration ... but we've only done it with one threshold, and we haven't tuned the parameters much. We won't go through all of the various ways we can fine-tune our models, but we can show how it is done: using loops.

<font color = 'red'>**Question 6. Write a loop that tries thresholds of .1, .3, .5, .7, and .9. Store the precision of each model at each threshold within their own arrays, named `knn5_precision`, `knn9_precision` and `knn15_precision`. Do the same for recall (replacing precision with recall in the naming convention).**</font>

The loop has been started below for you.

In [ ]:
threshold_values = make_array(0.1, 0.3, 0.5, 0.7, 0.9)
knn5_recall = make_array()
knn9_recall = make_array()
knn15_recall = make_array()

knn5_precision = make_array()
knn9_precision = make_array()
knn15_precision = make_array()

for threshold in make_array(0.1, 0.3, 0.5, 0.7, 0.9):
    pred5 = ...
    pred9 = ...
    pred15 = ...
    
    knn5_recall = ...
    knn9_recall = ...
    knn15_recall = ...
        
    knn5_precision = ...
    knn9_precision = ...
    knn15_precision = ...
    

Use the code below to look at the results afterwards.

In [ ]:
# You can use these to look at results
precision_results = Table().with_columns('Threshold', make_array(.1, .3, .5, .7, .9),
                              'K = 5', knn5_precision,
                              'K = 9', knn9_precision,
                              'K = 15', knn15_precision)

recall_results = Table().with_columns('Threshold', make_array(.1, .3, .5, .7, .9),
                              'K = 5', knn5_recall,
                              'K = 9', knn9_recall,
                              'K = 15', knn15_recall)

In [ ]:
precision_results

In [ ]:
recall_results

<font color = 'red'>**Question 7. What model and threshold combination gives the best precision? The best recall? If there are ties, choose the one that has the better performance in the other metric (so, for example, if two model/threshold combinations are tied for best recall, choose the one with the better precision of the two. If they are tied in both, then just choose whichever one you want.).**</font>

#### Step 6: Model Selection and Conclusions

Generally, when deciding on the best model, we compare the models we fit with each other, as well as against a baseline.

<font color = 'red'>**Question 8. Consider the threshold used in the best model by precision chosen in Question 6. If we were to try to predict whether a hand movement was "horizontal straight-line" or not completely randomly, how often would we be right? That is, what would be our precision if we were guessing completely randomly? How well does our best model perform compared to that baseline?**</font>